# NIS Clinical Exploration Template

In [ ]:
import pandas as pd
import copy
import os
import re

import statsmodels.api as sm
import numpy as np

import time

start_time = time.time() # for timing the execution of the entire notebook 

## Global Variables

Cervical spine example provided below.

### NIS

In [ ]:
%%time

patient_columns = ['AGE', # ensure AGE is first in list 
                   'FEMALE', 
                   'RACE',
                   'YEAR',
                   'PL_NCHS',
                   'ZIPINC_QRTL',
                   'HOSP_REGION', 
                   'PAY1', 
                   'TOTCHG', # -> {0,1} (<=avg_totchg, >avg_totchg)
                   'LOS', # -> {1,2,3,4,5,6,7+} 
                   'PRDAY1', # -> {-1,0,1} (before, during, after) admission 
                   'DISPUNIFORM'
                  ]

# list of all NIS comorbidities
comorbidity_columns = ['CMR_AIDS',
                         'CMR_ALCOHOL',
                         'CMR_AUTOIMMUNE',
                         'CMR_CANCER_LYMPH',
                         'CMR_CANCER_LEUK',
                         'CMR_CANCER_METS',
                         'CMR_CANCER_NSITU',
                         'CMR_CANCER_SOLID',
                         'CMR_DEMENTIA',
                         'CMR_DEPRESS',
                         'CMR_DIAB_UNCX',
                         'CMR_DIAB_CX',
                         'CMR_DRUG_ABUSE',
                         'CMR_HTN_CX',
                         'CMR_HTN_UNCX',
                         'CMR_LUNG_CHRONIC',
                         'CMR_OBESE',
                         'CMR_PERIVASC',
                         'CMR_THYROID_HYPO',
                         'CMR_THYROID_OTH'
                        ]


# List comprehension for all 'I10_DX' columns (1 to 40) and primary procedure column 
i10_columns = [f'I10_PR{n}' for n in range(1, 25)] + [f'I10_DX{n}' for n in range(1, 40+1)]

# Combine the two lists to form the complete list of selected columns
selected_columns = patient_columns + comorbidity_columns + i10_columns 

nis_df = pd.read_parquet("nis-data.parquet", columns=selected_columns)


In [ ]:
# use to check entry of diagnosis code in nis data 
from difflib import SequenceMatcher

def get_nis_df(year):
    nis_year = nis_df[nis_df['YEAR']==year]
    cols = ['YEAR'] + [f'I10_DX{n}' for n in range(1, 40+1)]
    nis_year = nis_year[cols]
    return nis_year

#data_2015 = get_nis_df(2015)
#data_2016 = get_nis_df(2016)
#data_2017 = get_nis_df(2017)
#data_2018 = get_nis_df(2018)
#data_2019 = get_nis_df(2019)

diagnosis_cols = [f'I10_DX{n}' for n in range(1, 2)]

# Function to check for similar values in given columns of a DataFrame
def similar_value_in_columns(df, columns, target_value, similarity_threshold=0.8):
    similar_values = []
    for column in columns:
        for index, value in df[column].items():  
            if SequenceMatcher(None, str(value), target_value).ratio() >= similarity_threshold:
                similar_values.append((index, column, value))
    return similar_values

#similar_values = similar_value_in_columns(data_2015, diagnosis_cols, 'M47.12')
#similar_values

### Research Specific Variables

#### Diganosis Exploration

In [ ]:
# Define the original codes
base_total_knee_codes = ['0SRC06', '0SRC07', '0SRC0J', '0SRC0K', '0SRD06', '0SRD07', '0SRD0J', '0SRD0K']
#base_partial_knee_codes = ['0SRC0L', '0SRC0M', '0SRC0N', '0SRD0L', '0SRD0M', '0SRD0N']

# Create modified versions with a period after the first three characters
modified_total_knee = [code[:3] + '.' + code[3:] for code in base_total_knee_codes]
#modified_partial_knee = [code[:3] + '.' + code[3:] for code in base_partial_knee_codes]

# Concatenate the base and modified lists
total_knee = base_total_knee_codes + modified_total_knee
#partial_knee = base_partial_knee_codes + modified_partial_knee

print("Total Knee Codes:", total_knee)
#print("Partial Knee Codes:", partial_knee)


In [ ]:
%%time

def determine_procedure_type(code):
    if code is None:
        return None
    if any(code.startswith(prefix) for prefix in total_knee):
        return 'total_knee'
    #if any(code.startswith(prefix) for prefix in partial_knee):
    #    return 'partial_knee'
    return None

# Apply the function to the entire Series instead of row by row
nis_df['Procedure_Type'] = nis_df['I10_PR1'].apply(determine_procedure_type)

# Calculate and print the frequencies
procedure_type_freq = nis_df['Procedure_Type'].value_counts()
print(procedure_type_freq)

In [ ]:
# primary diagnosis of interest (with variant for 2016)
diagnosis_code = 'M17.0' 
diagnosis_codes = ['M17.0', 'M170']

# Combine the original and modified lists
#procedure_codes = ["total_knee", "partial_knee"]
procedure_codes = ["total_knee"]

# List of complication code prefixes
complication_code_prefixes = ['M96', 'G97', 'I97', "M66", 'T81', 'T82', 'T84', 'T85', 'T88'] 

# variables needing to be condensed into smaller bins
condensed = ['AGE', 'TOTCHG', 'LOS', 'PRDAY1'] # use an empty list if no condensing is required

# Names
researcher = '' 
data_folder_name = '_data' # default
table_1_name = "table_1.csv" # default
table_2_name = "table_2.csv" # default
table_3_name = "table_3.csv" # default
table_4_name = "table_4.csv" # default
table_5_name = "table_5.csv" # default

# Ensure the directory exists/ create folder 
os.makedirs(data_folder_name, exist_ok=True)

### Table 3 Variables

In [ ]:
dependent_var = 'complications'

demographic_independent_variables = sorted(patient_columns)
#demographic_reference_variables.keys = demographic_independent_variables
#demographic_reference_variables.values = [] 

### Table 4 Variables

In [ ]:
table_4_adjusted = ['FEMALE', 'AGE', 'RACE', 'ZIPINC_QRTL'] # demographic variables to adjust for 

### Table 5 variables

In [ ]:
# 1. Compartment
base_bilateral = ['0SR.C06', '0SRC.07', '0SR.C0J', '0SR.C0K', '0SR.D06', '0SR.D07', '0SR.D0J', '0SR.D0K', '0SRC06', '0SRC07', '0SRC0J', '0SRC0K', '0SRD06', '0SRD07', '0SRD0J', '0SRD0K'] 
base_unicondylar = ['0SR.C0L', '0SR.C0M', '0SR.D0L', '0SR.D0M','0SRC0L', '0SRC0M', '0SRD0L', '0SRD0M']
base_patellofemoral = ['0SR.C0N', '0SR.D0N', '0SRC0N', '0SRD0N']

# 2. Cementation status
base_cemented = ['0SRC069', '0SRC0J9', '0SRC0L9', '0SRC0M9', '0SRC0N9', '0SRD069', '0SRD0J9', '0SRD0L9', '0SRD0M9', '0SRD0N9','0SR.C069', '0SR.C0J9', '0SR.C0L9', '0SR.C0M9', '0SR.C0N9', '0SR.D069', '0SR.D0J9', '0SR.D0L9', '0SR.D0M9', '0SR.D0N9']
base_uncemented = ['0SR.C06A', '0SR.C0JA', '0SR.C0LA', '0SR.C0MA', '0SR.C0NA', '0SR.D06A', '0SR.D0JA', '0SR.D0LA', '0SR.D0MA', '0SR.D0NA']

# 3. Tissue substitute
base_synthetic = ['0SRC06A', '0SRC0JA', '0SRC0LA', '0SRC0MA', '0SRC0NA', '0SRD06A', '0SRD0JA', '0SRD0LA', '0SRD0MA', '0SRD0NA','0SR.C06', '0SR.C0J', '0SR.C0L', '0SR.C0M', '0SR.C0N', '0SR.D06', '0SR.D0J', '0SR.D0L', '0SR.D0M', '0SR.D0N']
base_autologous = ['0SRC07', '0SRD07','0SR.C07', '0SR.D07']
base_non_autologous = ['0SRC0K', '0SRD0K','0SR.C0K', '0SR.D0K']

# 4. Articulating spacer
base_articulating_spacer_yes = ['0SRC0E', '0SRD0E','0SR.C0E', '0SR.D0E']

# Function to remove '.' from codes and add them to the base list
def modify_and_combine(base_list):
    modified_list = [code.replace('.', '') for code in base_list]
    return base_list + modified_list

# 1. Compartment
bilateral = modify_and_combine(base_bilateral)
unicondylar = modify_and_combine(base_unicondylar)
patellofemoral = modify_and_combine(base_patellofemoral)

# 2. Cementation status
cemented = modify_and_combine(base_cemented)
uncemented = modify_and_combine(base_uncemented)

# 3. Tissue substitute
synthetic = modify_and_combine(base_synthetic)
autologous = modify_and_combine(base_autologous)
non_autologous = modify_and_combine(base_non_autologous)

# 4. Articulating spacer
articulating_spacer_yes = modify_and_combine(base_articulating_spacer_yes)

surgical_variables = ['bilateral',
                       'unicondylar',
                       'patellofemoral'
                      ]

surgical_variables_2 = ['cemented',
                         'uncemented'
                        ]

surgical_variables_3 = ['synthetic',
                         'autologous',
                        'non_autologous'
                        ]

surgical_variables_4 = ['articulating_spacer_yes'
                        ]

surgical_independent_variables_with_reference = surgical_variables + surgical_variables_2 + surgical_variables_3 + surgical_variables_4
surgical_independent_variables = surgical_independent_variables_with_reference

table_5_adjusted = ['comorbidity', 'FEMALE', 'AGE', 'RACE', 'ZIPINC_QRTL'] # comorbidity + demographic variables to adjust for 

See [link](https://hcup-us.ahrq.gov/db/nation/nis/nisdde.jsp) for full data description.

## Set Up

In [ ]:
def create_df_snapshot(df):
    """
    Creates a deep copy of the provided DataFrame to be used as a snapshot for comparison.
    """
    return copy.deepcopy(df)

def assert_unchanged(original_df_snapshot, current_df):
    """
    Asserts that the current DataFrame is identical to the original snapshot.
    """
    msg="DataFrame has been altered"
    pd.testing.assert_frame_equal(original_df_snapshot, current_df, 
                                  check_dtype=True, 
                                  check_exact=True, 
                                  check_like=True, 
                                  check_datetimelike_compat=True, 
                                  check_categorical=True, 
                                  rtol=0, 
                                  atol=0, 
                                  obj=msg)

### Diagnosis Group

In [ ]:
# filtering based on primary diagnosis (This is the universe of discharges that we are concerned with)
#diagnosis_df = nis_df[nis_df['I10_DX1'] == diagnosis_code].copy()
# Filter the DataFrame for rows where 'I10_DX1' matches any of the codes in the list
diagnosis_df = nis_df[nis_df['I10_DX1'].isin(diagnosis_codes)].copy()

diagnosis_df = diagnosis_df[diagnosis_df['AGE'] >= 35]

diagnosis_df_snapshot = create_df_snapshot(diagnosis_df) # record instantiation
diagnosis_df

### Procedure Group

In [ ]:
#procedure_df = diagnosis_df[diagnosis_df['I10_PR1'].isin(procedure_codes)]
# fill nan values with empty string before searching to avoid error
procedure_df = diagnosis_df[diagnosis_df['Procedure_Type'].fillna('').isin(procedure_codes)]

# Dictionary to hold the dataframes for each procedure code (if needed)
procedure_dfs = {}

# Loop through each code in the procedure_codes list
for i, code in enumerate(procedure_codes, start=1):
    # Filter diagnosis_df for the current procedure code and assign to the dictionary
    procedure_dfs[f'procedure{i}_df'] = diagnosis_df[diagnosis_df['Procedure_Type'] == code]
    
# Assign each DataFrame in procedure_dfs to a global variable with the same name
for df_name, df in procedure_dfs.items():
    globals()[df_name] = df

print(f"defined: {list(procedure_dfs.keys())}") # Now, procedure1_df, procedure2_df, ... are your DataFrames

# complement of procedure_df wrt diagnosis_df
non_procedure_df = diagnosis_df.drop(procedure_df.index) 

def add_complication_column(df):
    df = df.copy()
    # List of columns to check for complication codes
    dx_columns = [f'I10_DX{n}' for n in range(2, 41)]
    
    # Function to check if any cell value starts with any of the complication prefixes
    def has_complication(row):
        for prefix in complication_code_prefixes:
            if any(row[col].startswith(prefix) for col in dx_columns if pd.notna(row[col])):
                return 1
        return 0

    # Apply the function to each row
    df['complications'] = df[dx_columns].apply(has_complication, axis=1)
    
    return df

# add complications column to procedure_df (will be used later for table_3)
procedure_df = add_complication_column(procedure_df)
procedure_df_snapshot = create_df_snapshot(procedure_df) # record instantiation
procedure_df

In [ ]:
procedure1_df

In [ ]:
print(f"For discharges with primary diagnosis {diagnosis_code}:")
print("")

avg_age = round(diagnosis_df["AGE"].mean())
std_age = round(diagnosis_df["AGE"].std())
print(f"Average age: {avg_age} years (standard deviation age: {std_age} years)")

avg_los = round(diagnosis_df["LOS"].mean())
print(f"Average length of stay: {avg_los} days")

avg_totchg = round(diagnosis_df["TOTCHG"].mean())
formatted_avg_totchg = f"${avg_totchg:,.0f} USD"
print(f"Average total charge: {formatted_avg_totchg}")

# Table 1: Table of Patient Characterstics

In [ ]:
assert_unchanged(procedure_df_snapshot, procedure_df)
assert_unchanged(diagnosis_df_snapshot, diagnosis_df)

In [ ]:
def init_patient_characteristics_table():
    global diagnosis_df, procedure_df, non_procedure_df

    # Ensure the dataframes are copied
    temp_proc_df = procedure_df.copy()
    temp_non_proc_df = non_procedure_df.copy()

    # Define new age bins and labels
    age_bins = [35, 50, 75, float('inf')]
    age_labels = ['35-49', '50-74', '75+']

    # Convert AGE to numeric and handle non-numeric values
    #temp_proc_df['AGE'] = pd.to_numeric(temp_proc_df['AGE'], errors='coerce')
    #temp_non_proc_df['AGE'] = pd.to_numeric(temp_non_proc_df['AGE'], errors='coerce')

    # Bin the AGE data
    temp_proc_df['AGE_bin'] = pd.cut(temp_proc_df['AGE'], bins=age_bins, labels=age_labels, right=False)
    temp_non_proc_df['AGE_bin'] = pd.cut(temp_non_proc_df['AGE'], bins=age_bins, labels=age_labels, right=False)

    # Initialize a table with columns 'procedure' and 'non-procedure'
    table = pd.DataFrame(columns=['procedure', 'non-procedure']).astype('object')

    # Total count
    table.loc['total count', 'procedure'] = f"{len(temp_proc_df)} ({len(temp_proc_df)/len(diagnosis_df):.2%})"
    table.loc['total count', 'non-procedure'] = f"{len(temp_non_proc_df)} ({len(temp_non_proc_df)/len(diagnosis_df):.2%})"

    # Age mean and standard deviation
    age_mean_std_procedure = f"{temp_proc_df['AGE'].mean():.2f}, {temp_proc_df['AGE'].std():.2f}"
    age_mean_std_nonprocedure = f"{temp_non_proc_df['AGE'].mean():.2f}, {temp_non_proc_df['AGE'].std():.2f}"
    table.loc['AGE (mean, std)', 'procedure'] = age_mean_std_procedure
    table.loc['AGE (mean, std)', 'non-procedure'] = age_mean_std_nonprocedure

    # Age bins count
    for label in age_labels:
        count_procedure = temp_proc_df[temp_proc_df['AGE_bin'] == label].shape[0]
        count_non_procedure = temp_non_proc_df[temp_non_proc_df['AGE_bin'] == label].shape[0]
        table.loc[f'AGE {label}', 'procedure'] = f"{count_procedure} ({count_procedure/len(temp_proc_df):.2%})"
        table.loc[f'AGE {label}', 'non-procedure'] = f"{count_non_procedure} ({count_non_procedure/len(temp_non_proc_df):.2%})"
    
    print(temp_proc_df)
    return table


def create_non_condensed_patient_characteristics_table():
    characteristics = [col for col in patient_columns[1:] if col not in condensed]

    table = init_patient_characteristics_table()
    for char in characteristics:
        for value in diagnosis_df[char].unique():
            count_procedure = procedure_df[procedure_df[char] == value].shape[0]
            count_non_procedure = non_procedure_df[non_procedure_df[char] == value].shape[0]

            if count_procedure > 0 and count_non_procedure > 0:
                table.loc[f'{char}_{value}', 'procedure'] = f"{count_procedure} ({count_procedure/len(procedure_df):.2%})"
                table.loc[f'{char}_{value}', 'non-procedure'] = f"{count_non_procedure} ({count_non_procedure/len(non_procedure_df):.2%})"

    return table


def add_LOS(table):
    # Temporary DataFrames with LOS binning
    temp_proc_df = procedure_df.copy()
    temp_non_proc_df = non_procedure_df.copy()

    # Define the bins and labels for the LOS
    bins = [0, 1, 2, 3, 4, 5, 6, 7, float('inf')]
    labels = ['0', '1', '2', '3', '4', '5', '6', '7+']

    # Bin the LOS data
    temp_proc_df['LOS_bin'] = pd.cut(temp_proc_df['LOS'], bins=bins, labels=labels, right=False)
    temp_non_proc_df['LOS_bin'] = pd.cut(temp_non_proc_df['LOS'], bins=bins, labels=labels, right=False)

    # Calculate counts and proportions for each bin
    for label in labels:
        count_procedure = temp_proc_df[temp_proc_df['LOS_bin'] == label].shape[0]
        count_non_procedure = temp_non_proc_df[temp_non_proc_df['LOS_bin'] == label].shape[0]

        if count_procedure > 0 and count_non_procedure > 0:
            table.loc[f'LOS_{label}', 'procedure'] = f"{count_procedure} ({count_procedure/len(temp_proc_df):.2%})"
            table.loc[f'LOS_{label}', 'non-procedure'] = f"{count_non_procedure} ({count_non_procedure/len(temp_non_proc_df):.2%})"

    return table

def add_LOS_table_3(table):
    local_df = procedure_df.copy()  # Work on a copy
    bins = [0, 1, 2, 3, 4, 5, 6, float('inf')]
    labels = ['0', '1', '2', '3', '4', '5', '6', '7+']
    local_df['LOS_bin'] = pd.cut(local_df['LOS'], bins=bins, labels=labels, right=False)

    for label in labels:
        subset_df = local_df[local_df['LOS_bin'] == label]
        count_procedure = subset_df.shape[0]
        count_complications = subset_df['complications'].sum()

        table.loc[f'LOS_{label}', 'procedure'] = f"{count_procedure} ({count_procedure/len(local_df):.2%})"
        table.loc[f'LOS_{label}', 'complication'] = f"{count_complications} ({count_complications/count_procedure:.2%})" if count_procedure > 0 else '0 (0.00%)'

    return table


def add_PRDAY1(table):
    # Temporary DataFrames with PRDAY1 categorization
    temp_proc_df = procedure_df.copy()
    temp_non_proc_df = non_procedure_df.copy()

    # Define a function to categorize PRDAY1 values
    def categorize_prday1(value):
        if value >= -4 and value <= -1:
            return 'before'
        elif value == 0:
            return 'during'
        elif value >= 1: # 
            return 'after'
        else:
            return 'other'

    # Apply categorization to PRDAY1
    temp_proc_df['PRDAY1_cat'] = temp_proc_df['PRDAY1'].apply(categorize_prday1)
    temp_non_proc_df['PRDAY1_cat'] = temp_non_proc_df['PRDAY1'].apply(categorize_prday1)

    # Calculate counts and proportions for each category
    for category in ['before', 'during', 'after', 'other']:
        count_procedure = temp_proc_df[temp_proc_df['PRDAY1_cat'] == category].shape[0]
        count_non_procedure = temp_non_proc_df[temp_non_proc_df['PRDAY1_cat'] == category].shape[0]

        if count_procedure > 0 and count_non_procedure > 0:
            table.loc[f'PRDAY1_{category}', 'procedure'] = f"{count_procedure} ({count_procedure/len(temp_proc_df):.2%})"
            table.loc[f'PRDAY1_{category}', 'non-procedure'] = f"{count_non_procedure} ({count_non_procedure/len(temp_non_proc_df):.2%})"

    return table

def add_TOTCHG(table):
    # Temporary DataFrames with TOTCHG categorization
    temp_proc_df = procedure_df.copy()
    temp_non_proc_df = non_procedure_df.copy()

    # Categorize TOTCHG based on the condition
    temp_proc_df['TOTCHG_cat'] = (temp_proc_df['TOTCHG'] > avg_totchg).astype(int)
    temp_non_proc_df['TOTCHG_cat'] = (temp_non_proc_df['TOTCHG'] > avg_totchg).astype(int)

    # Calculate counts and proportions for each category (0 and 1)
    for category in [0, 1]:
        count_procedure = temp_proc_df[temp_proc_df['TOTCHG_cat'] == category].shape[0]
        count_non_procedure = temp_non_proc_df[temp_non_proc_df['TOTCHG_cat'] == category].shape[0]

        if count_procedure > 0 and count_non_procedure > 0:
            table.loc[f'TOTCHG_{category}', 'procedure'] = f"{count_procedure} ({count_procedure/len(temp_proc_df):.2%})"
            table.loc[f'TOTCHG_{category}', 'non-procedure'] = f"{count_non_procedure} ({count_non_procedure/len(temp_non_proc_df):.2%})"

    return table


def create_patient_characteristics_table():
    # Initialize the table
    table = create_non_condensed_patient_characteristics_table() # calls init_patient_characteristics_table

    # Add condensed variables based on the condensed list
    for condensed_variable in condensed:
        if condensed_variable == 'TOTCHG':
            table = add_TOTCHG(table)
        elif condensed_variable == 'LOS':
            table = add_LOS(table)
        elif condensed_variable == 'PRDAY1':
            table = add_PRDAY1(table)

    return table

In [ ]:
def check_percentages_sum_to_100(table, tolerance = 0.1):
    # Function to extract percentage value from a string
    def extract_percentage(s):
        match = re.search(r"(\d+\.\d+)%", s)
        return float(match.group(1)) if match else 0

    # List to store error messages
    errors = []

    # Check 'total count' row first
    if 'total count' in table.index:
        total_count_procedure = extract_percentage(table.loc['total count', 'procedure'])
        total_count_non_procedure = extract_percentage(table.loc['total count', 'non-procedure'])
        if not (97 <= (total_count_procedure + total_count_non_procedure) <= 100.1):  # Allowing a tiny margin for floating-point arithmetic
            errors.append("Total counts in 'procedure' and 'non-procedure' do not sum to approximately 100%")

    # Initialize a dictionary to hold the sum of percentages for each variable and category
    percentage_sums = {}

    # Iterate through each cell in the table to sum the percentages
    for index, row in table.iterrows():
        # Skip 'total count' and 'AGE (mean, std)' rows
        if index in ['total count', 'AGE (mean, std)']:
            continue

        # Extract variable name (e.g., 'FEMALE' from 'FEMALE_0.0')
        var_name = index.split('_')[0]

        # Sum percentages for each variable across its categories
        for col in table.columns:
            val = extract_percentage(row[col])
            if var_name not in percentage_sums:
                percentage_sums[var_name] = {col: 0}
            if col not in percentage_sums[var_name]:
                percentage_sums[var_name][col] = 0
            percentage_sums[var_name][col] += val

    # Check if the sums are approximately 100% for each variable in each column
    #tolerance = 0.1  # e.g., 0.1% tolerance
    for var, cols in percentage_sums.items():
        for col, total in cols.items():
            if not (100 - tolerance <= total <= 100 + tolerance):
                errors.append(f"Variable '{var}' in column '{col}' does not sum to approximately 100% (Sum: {total}%)")

    # Check if any errors were found
    if errors:
        return False, "\n".join(errors)
    else:
        return True, "All checks passed: Total counts and variables sum to approximately 100% per column"

In [ ]:
table_1 = create_patient_characteristics_table()
table_1

In [ ]:
result, message = check_percentages_sum_to_100(table_1, 0.1)
print(result, message)

In [ ]:
# Saving the DataFrame as a CSV file
table_1.to_csv(f"{data_folder_name}/{researcher}_{table_1_name}")

# Table 2: Complication Analysis

In [ ]:
assert_unchanged(procedure_df_snapshot, procedure_df)
assert_unchanged(diagnosis_df_snapshot, diagnosis_df)

In [ ]:
def find_complication_codes(dataframe=diagnosis_df, start_col=2, end_col=40):
    # Generate the column names based on the provided start and end columns
    column_names = [f'I10_DX{i}' for i in range(start_col, end_col + 1)]
    # Initialize an empty set to store unique codes across all columns
    unique_codes_set = set()
    
    # Iterate over each diagnosis column and find unique codes that start with the given prefix
    for column in column_names:
        unique_values = dataframe[column].dropna().unique()
        # Update the unique codes set with codes that start with the prefix
        for prefix in complication_code_prefixes:
            unique_codes_set.update(code for code in unique_values if str(code).startswith(prefix))
    
    # Return the unique codes as a sorted list
    return sorted(unique_codes_set)

# Example of how to use the refactored function:
complications_codes = find_complication_codes()

In [ ]:
procedure1_df 

In [ ]:
def get_complication_counts(df, name):
    dx_columns = [f'I10_DX{n}' for n in range(2, 41)]
    complication_counts_per_prefix = {}

    df_length = len(df)
    
    for prefix in complication_code_prefixes:
        count = df[dx_columns].apply(lambda row: any(row.str.startswith(prefix, na=False)), axis=1).sum()
        if count >= 10:
            proportion = 100 * count / df_length if df_length > 0 else 0
            complication_counts_per_prefix[prefix] = (count, round(proportion, 2))

    total_complications = sum(count for count, _ in complication_counts_per_prefix.values())
    complication_counts_per_prefix['Total'] = (total_complications, round(100 * total_complications / df_length, 2) if df_length > 0 else 0)

    return complication_counts_per_prefix


def get_complications_table():
    # Assuming complication_code_prefixes is defined globally
    complication_summary_df = pd.DataFrame()

    diagnosis_data = get_complication_counts(diagnosis_df, "diagnosis")
    procedure_data = get_complication_counts(procedure_df, "procedure")
    
    # Determine the common complications across both datasets
    common_complications = set(diagnosis_data).intersection(set(procedure_data))
    complications_table_index = list(common_complications) + ['Total']

    complication_summary_df['Diagnosis'] = [f"{diagnosis_data[prefix][0]} ({diagnosis_data[prefix][1]}%)" for prefix in complications_table_index]
    complication_summary_df['Procedure'] = [f"{procedure_data[prefix][0]} ({procedure_data[prefix][1]}%)" for prefix in complications_table_index]

    if 'procedure_dfs' in globals() and isinstance(procedure_dfs, dict):
        for df_name, df in procedure_dfs.items():
            df_data = get_complication_counts(df, df_name)
            complication_summary_df[df_name] = [f"{df_data[prefix][0]} ({df_data[prefix][1]}%)" if prefix in df_data else "0 (0%)" for prefix in complications_table_index]

    complication_summary_df.index = complications_table_index
    return complication_summary_df




In [ ]:
table_2 = get_complications_table()
table_2

In [ ]:
def extract_count(value):
    # Extracts numerical count from the cell value
    match = re.match(r"(\d+)", value)
    return int(match.group(1)) if match else 0

def test_complication_counts(table):
    # Summing counts from all 'procedureX_df' columns, excluding 'Procedure'
    procedure_totals = sum([sum(table[col].apply(extract_count)) for col in table.columns if col.startswith('proc') and col != 'Procedure'])

    # Extract total counts for Procedure and Diagnosis
    procedure_total_count = extract_count(table.loc['Total', 'Procedure'])
    diagnosis_total_count = extract_count(table.loc['Total', 'Diagnosis'])

    # Check if the sums are as expected
    assert procedure_totals == procedure_total_count, "Sum of procedure counts does not match total procedure count"
    assert procedure_total_count < diagnosis_total_count, "Total procedure count is not less than total diagnosis count"

    return "Test passed"

# Assuming 'table' is your DataFrame
# test_complication_counts(table)


#test_complication_counts(table_2)

In [ ]:
table_2.to_csv(f"{data_folder_name}/{researcher}_{table_2_name}")

# Table 3: Demographic variables

In [ ]:
assert_unchanged(procedure_df_snapshot, procedure_df)
assert_unchanged(diagnosis_df_snapshot, diagnosis_df)

In [ ]:
def add_AGE_table_3(table):
    local_df = procedure_df.copy()  # Work on a copy

    # Define new age bins and labels
    bins = [35,50, 75, float('inf')]
    labels = ['35-49','50-74', '75+']
    local_df['AGE_bin'] = pd.cut(local_df['AGE'], bins=bins, labels=labels, right=False)

    for label in labels:
        subset_df = local_df[local_df['AGE_bin'] == label]
        count_procedure = subset_df.shape[0]
        count_complications = subset_df['complications'].sum()

        # Calculate and format the statistics
        proc_stat = f"{count_procedure} ({count_procedure/len(local_df):.2%})"
        comp_stat = f"{count_complications} ({count_complications/count_procedure:.2%})" if count_procedure > 0 else '0 (0.00%)'

        # Add statistics to the table
        table.loc[f'AGE_{label}', 'procedure'] = proc_stat
        table.loc[f'AGE_{label}', 'complication'] = comp_stat

    return table

def add_LOS_table_3(table):
    local_df = procedure_df.copy()  # Work on a copy
    bins = [0, 1, 2, 3, 4, 5, 6, 7, float('inf')]
    labels = ['0', '1', '2', '3', '4', '5', '6', '7+']
    local_df['LOS_bin'] = pd.cut(local_df['LOS'], bins=bins, labels=labels, right=False)

    for label in labels:
        subset_df = local_df[local_df['LOS_bin'] == label]
        count_procedure = subset_df.shape[0]
        count_complications = subset_df['complications'].sum()

        table.loc[f'LOS_{label}', 'procedure'] = f"{count_procedure} ({count_procedure/len(local_df):.2%})"
        table.loc[f'LOS_{label}', 'complication'] = f"{count_complications} ({count_complications/count_procedure:.2%})" if count_procedure > 0 else '0 (0.00%)'

    return table

def add_PRDAY1_table_3(table):
    local_df = procedure_df.copy()  # Work on a copy
    def categorize_prday1(value):
        if value >= -4 and value <= -1:
            return 'before'
        elif value == 0:
            return 'during'
        elif value >= 1:
            return 'after'
        else:
            return 'other'

    local_df['PRDAY1_cat'] = local_df['PRDAY1'].apply(categorize_prday1)

    for category in ['before', 'during', 'after', 'other']:
        subset_df = local_df[local_df['PRDAY1_cat'] == category]
        count_procedure = subset_df.shape[0]
        count_complications = subset_df['complications'].sum()

        table.loc[f'PRDAY1_{category}', 'procedure'] = f"{count_procedure} ({count_procedure/len(local_df):.2%})"
        table.loc[f'PRDAY1_{category}', 'complication'] = f"{count_complications} ({count_complications/count_procedure:.2%})" if count_procedure > 0 else '0 (0.00%)'

    return table


def add_TOTCHG_table_3(table):
    local_df = procedure_df.copy()  # Work on a copy
    local_df['TOTCHG_cat'] = (local_df['TOTCHG'] > avg_totchg).astype(int)

    for category in [0, 1]:
        subset_df = local_df[local_df['TOTCHG_cat'] == category]
        count_procedure = subset_df.shape[0]
        count_complications = subset_df['complications'].sum()

        table.loc[f'TOTCHG_{category}', 'procedure'] = f"{count_procedure} ({count_procedure/len(local_df):.2%})"
        table.loc[f'TOTCHG_{category}', 'complication'] = f"{count_complications} ({count_complications/count_procedure:.2%})" if count_procedure > 0 else '0 (0.00%)'

    return table


def add_complication_summary(table, procedure_df):
    """
    Add complication summary statistics to a table.

    Parameters:
    - table (pd.DataFrame): The table to which the summary statistics will be added.
    - procedure_df (pd.DataFrame): The DataFrame containing procedure data, including complications.

    Returns:
    - pd.DataFrame: The updated table with complication summary statistics.
    """
    total_complications = procedure_df['complications'].sum()
    table.loc['total count', 'complication'] = f"{total_complications} ({total_complications/len(procedure_df):.2%})"
    complication_age_mean = procedure_df[procedure_df['complications'] == 1]['AGE'].mean()
    complication_age_std = procedure_df[procedure_df['complications'] == 1]['AGE'].std()
    table.loc['AGE (mean, std)', 'complication'] = f"{complication_age_mean:.2f}, {complication_age_std:.2f}"
    return table

def add_patient_characteristics(table, procedure_df, characteristics):
    """
    Add patient characteristics to a table.

    Parameters:
    - table (pd.DataFrame): The table to which the patient characteristics will be added.
    - procedure_df (pd.DataFrame): The DataFrame containing procedure data.
    - characteristics (list): A list of characteristics to be included.

    Returns:
    - pd.DataFrame: The updated table with patient characteristics.
    """
    characteristics = [col for col in patient_columns[1:] if col not in condensed] # non-condensed variables
    
    for col in characteristics:
        for value in procedure_df[col].unique():
            count_procedure = procedure_df[procedure_df[col] == value].shape[0]
            if count_procedure > 0:
                table.loc[f'{col}_{value}', 'procedure'] = f"{count_procedure} ({count_procedure/len(procedure_df):.2%})"
    return table

def add_complication_data(table, procedure_df, characteristics):
    """
    Add complication data to a table.

    Parameters:
    - table (pd.DataFrame): The table to which the complication data will be added.
    - procedure_df (pd.DataFrame): The DataFrame containing procedure data, including complications.
    - characteristics (list): A list of characteristics to consider for complication data.

    Returns:
    - pd.DataFrame: The updated table with complication data.
    """
    for col in characteristics:
        complication_counts = procedure_df.groupby(col)['complications'].sum()
        total_counts = procedure_df[col].value_counts()
        complication_proportion = complication_counts / total_counts

        for value, count in complication_counts.items():
            if total_counts.get(value, 0) > 0:
                proportion = complication_proportion.get(value, 0)
                table.loc[f'{col}_{value}', 'complication'] = f"{count} ({proportion:.2%})"
    return table

def get_table_3_counts(procedure_df, condensed):
    """
    Create a table with patient characteristics and complication information.

    Parameters:
    - procedure_df (pd.DataFrame): The DataFrame containing procedure data, including complications.
    - condensed (list): A list of condensed variables to consider.

    Returns:
    - pd.DataFrame: The table containing patient characteristics and complication information.
    """
    table = pd.DataFrame(columns=['procedure', 'complication']).astype('object')
    table.loc['total count', 'procedure'] = f"{len(procedure_df)} ({len(procedure_df)/len(procedure_df):.2%})"
    age_mean_std_procedure = f"{procedure_df['AGE'].mean():.2f}, {procedure_df['AGE'].std():.2f}"
    table.loc['AGE (mean, std)', 'procedure'] = age_mean_std_procedure
    
    characteristics = [col for col in patient_columns[1:] if col not in condensed] # non-condensed variables

    table = add_complication_summary(table, procedure_df, )
    table = add_patient_characteristics(table, procedure_df, characteristics)
    table = add_complication_data(table, procedure_df, characteristics)

    for condensed_variable in condensed:
        if condensed_variable == 'TOTCHG':
            table = add_TOTCHG_table_3(table)
        elif condensed_variable == 'LOS':
            table = add_LOS_table_3(table)
        elif condensed_variable == 'PRDAY1':
            table = add_PRDAY1_table_3(table)
        elif condensed_variable == 'AGE':
            table = add_AGE_table_3(table)

    # Split the table into the first row and the rest
    first_row = table.iloc[:2]
    rest_of_table = table.iloc[2:]

    # Sort the rest of the table based on the index (row labels) in ascending order
    rest_of_table_sorted = rest_of_table.copy().sort_index()

    # Concatenate the first row with the sorted rest of the table
    sorted_table = pd.concat([first_row, rest_of_table_sorted])

    return sorted_table

    
table_3_counts = get_table_3_counts(procedure_df, condensed)
table_3_counts

In [ ]:
assert_unchanged(procedure_df_snapshot, procedure_df)
assert_unchanged(diagnosis_df_snapshot, diagnosis_df)

In [ ]:
def condense_LOS(df):
    # Create a copy of the DataFrame to avoid modifying the original DataFrame
    df_copy = df.copy()

    # Define the bins and labels for the LOS
    bins = [0, 1, 2, 3, 4, 5, 6, 7, float('inf')]
    labels = ['0', '1', '2', '3', '4', '5', '6', '7+']  # Including '0' as a category for LOS value of 0

    # Apply the pd.cut function to categorize LOS
    df_copy['LOS'] = pd.cut(df_copy['LOS'], bins=bins, labels=labels, right=False)

    # Handle NaN values if necessary
    # df_copy['LOS_cat'] = df_copy['LOS_cat'].cat.add_categories('NaN').fillna('NaN')

    return df_copy

def condense_PRDAY1(df):
    def categorize_prday1(value):
        if value >= -4 and value <= -1:
            return 'before' # before admission 
        elif value == 0:
            return 'during' # during admission
        elif value >= 1:
            return 'after' # after admission
        else:
            return 'other' # other 

    df.loc[:, 'PRDAY1'] = df['PRDAY1'].apply(categorize_prday1)
    return df


def condense_TOTCHG(df):
    #avg_totchg = round(diagnosis_df['TOTCHG'].mean())
    df.loc[:, 'TOTCHG'] = (df['TOTCHG'] > avg_totchg).astype(int)
    return df

#def condense_AGE(df):
 #   df.loc[:, 'AGE'] = (df['AGE'] > avg_age).astype(int)
  #  return df

def condense_AGE(df):
    # Convert AGE to numeric, handling non-numeric entries as NaN
    df['AGE'] = pd.to_numeric(df['AGE'], errors='coerce')

    # Define adjusted age bins and labels
    bins = [35,50, 75, float('inf')]
    labels = ['35-49','50-74', '75+']
    df['AGE'] = pd.cut(df['AGE'], bins=bins, labels=labels, right=False)

    return df


def get_odd_ratios_table_3():
    or_df = procedure_df[demographic_independent_variables+['complications']]
    or_df = condense_LOS(or_df)
    or_df = condense_PRDAY1(or_df)
    or_df = condense_TOTCHG(or_df)
    or_df = condense_AGE(or_df)
    old_len = len(or_df)
    or_df = or_df.dropna()
    new_len = len(or_df)
    print(f"dropped {old_len - new_len} discharges with missing data")

    or_df = pd.get_dummies(or_df, columns=or_df.columns.drop('complications'))

    # Sort the columns in alphabetical order while keeping 'complications' as the first column
    sorted_columns = sorted(or_df.columns.drop('complications'))
    or_df = or_df[['complications'] + sorted_columns]
    
    return or_df

or_df3 = get_odd_ratios_table_3()


def get_demographic_reference_variables(df):
    # Check if 'demographic_reference_variables' exists globally
    if 'demographic_reference_variables' in globals():
        return globals()['reference_variables']

    # Identify all unique category prefixes (considering the part before the last underscore)
    unique_categories = set('_'.join(col.split('_')[:-1]) for col in df.columns if '_' in col)

    # Initialize a dictionary to store reference variables for each category
    reference_variables = {}

    for category in unique_categories:
        # Find the first dummy variable of each category to use as the reference
        reference_var = next((col for col in df.columns if col.startswith(category + '_')), None)
        
        if reference_var:
            reference_variables[category] = reference_var

    return reference_variables

def get_demographic_reference_variables(df):
    # Check if 'demographic_reference_variables' exists globally
    if 'demographic_reference_variables' in globals():
        return globals()['reference_variables']

    # Identify all unique category prefixes (considering the part before the last underscore)
    unique_categories = set('_'.join(col.split('_')[:-1]) for col in df.columns if '_' in col)

    # Initialize a dictionary to store reference variables for each category
    reference_variables = {}

    for category in unique_categories:
        if category == 'AGE':
            # Explicitly set the reference variable for AGE 
            reference_var = f'{category}_35-49'
            #reference_var = f'{category}_65+'
        else:
            # For other categories, find the first dummy variable as the reference
            reference_var = next((col for col in df.columns if col.startswith(category + '_')), None)
        
        if reference_var:
            reference_variables[category] = reference_var

    return reference_variables

reference_vars = get_demographic_reference_variables(or_df3)
print("Reference variables for each category:")
for category, ref_var in reference_vars.items():
    print(f"{category}: {ref_var}")

or_df3

In [ ]:
def check_columns(df):
    all_numeric = True
    no_missing_values = True
    problematic_columns = []

    # Check each column for data type and missing values
    for col in df.columns:
        dtype = df[col].dtype
        missing_values = df[col].isnull().sum()

        if dtype == 'object' or missing_values > 0:
            # If any column is non-numeric or has missing values, record this
            all_numeric = False if dtype == 'object' else all_numeric
            no_missing_values = False if missing_values > 0 else no_missing_values

            # Store information about problematic columns
            dtype_info = "Not Numeric" if dtype == 'object' else "Numeric"
            missing_info = f"Missing Values: {missing_values}" if missing_values > 0 else "No Missing Values"
            problematic_columns.append(f"Column: {col}, Type: {dtype_info}, {missing_info}")

    # Print statements based on the checks
    if all_numeric and no_missing_values:
        print("All types numeric, no missing values")
    else:
        for info in problematic_columns:
            print(info)

check_columns(or_df3)

In [ ]:
def get_table_3_logistic_regression(df, dependent_var, independent_vars_prefixes, reference_vars):
    results = {}
    for prefix in independent_vars_prefixes:
        try:
            # Filter variables with the same prefix, excluding the reference variable
            vars_with_prefix = [col for col in df.columns if col.startswith(prefix) and col != reference_vars[prefix]]

            # Define X and y
            X = df[vars_with_prefix].astype(float)
            y = df[dependent_var].astype(float)

            # Add constant to the model
            X = sm.add_constant(X)

            # Fit logistic regression model
            model = sm.Logit(y, X)
            fitted_model = model.fit()

            # Calculating Odds Ratios, Confidence Intervals, and P-values
            for var in vars_with_prefix:
                odds_ratio = np.exp(fitted_model.params[var]).round(2)
                conf = fitted_model.conf_int().loc[var]
                ci_lower, ci_upper = np.exp(conf[0]).round(2), np.exp(conf[1]).round(2)
                p_value = fitted_model.pvalues[var].round(3)

                # Store results
                results[var] = {'OR': odds_ratio, '2.5%': ci_lower, '97.5%': ci_upper, 'p-value': p_value}
        except Exception as e:
            print(f"Error with variable prefix {prefix}: {e}")

    return pd.DataFrame(results).T


logistic_regression_results_3 = get_table_3_logistic_regression(or_df3, 
                                                                dependent_var, # globally defined 
                                                                demographic_independent_variables, # globally_defined
                                                                reference_vars)

# Transform the DataFrame
logistic_regression_results_3['OR (95% CI)'] = logistic_regression_results_3.apply(
    lambda row: f"{row['OR']} ({row['2.5%']} - {row['97.5%']})", axis=1
)
logistic_regression_results_3 = logistic_regression_results_3[['OR (95% CI)', 'p-value']]

# Display the transformed DataFrame
logistic_regression_results_3

Single Multivariable Logistic Regression Model:

Approach: Fit one logistic regression model including all demographic variables as independent variables. This model assesses the effect of each variable on the outcome while controlling for the effects of the other variables.
Odds Ratios: The ORs obtained from this model reflect the effect of each demographic variable on the outcome, adjusted for the influence of other variables in the model. This is important if there are potential confounders or if the variables are correlated.
Interpretation: This approach provides a more holistic understanding of how each variable contributes to the outcome in the presence of other variables. It's more appropriate when the goal is to understand the effect of each variable in a real-world, multivariable context.

In summary, for Table 3, if the aim is to understand the independent contribution of each demographic variable to the risk of complications, considering the potential interplay and confounding effects of other variables, a single multivariable logistic regression model would be more appropriate. This approach provides adjusted odds ratios that offer a more comprehensive view of each variable's effect in the context of others.

In [ ]:
assert_unchanged(procedure_df_snapshot, procedure_df)
assert_unchanged(diagnosis_df_snapshot, diagnosis_df)

In [ ]:
def get_Table_3():

    # Convert the index of logistic_regression_results to a column if it's not already
    logistic_regression_results_3.reset_index(inplace=True)
    logistic_regression_results_3.rename(columns={'index': 'variable'}, inplace=True)

    # Merge the tables
    # This assumes that the rows in table_3_counts are named exactly as the variables in logistic_regression_results
    table_3 = table_3_counts.merge(logistic_regression_results_3, left_index=True, right_on='variable', how='left')

    # If needed, set 'variable' column as the new index
    table_3.set_index('variable', inplace=True)
    return table_3

table_3 = get_Table_3()
table_3 

In [ ]:
table_3.to_csv(f"{data_folder_name}/{researcher}_{table_3_name}")

# Table 4: Medical comorbidities

In [ ]:
assert_unchanged(procedure_df_snapshot, procedure_df)
assert_unchanged(diagnosis_df_snapshot, diagnosis_df)

In [ ]:
def get_table_4_counts(procedure_df):
    # Initialize a list to store the results
    results = []

    # Function to format count and proportion
    def format_count_proportion(count, total):
        proportion = (count / total * 100) if total != 0 else 0
        return f"{count} ({proportion:.2f}%)"

    # Iterate over each comorbidity column
    for col in comorbidity_columns:
        # Calculate the counts for the comorbidity
        total_count = procedure_df[col].value_counts()

        # Calculate the counts for complications within each comorbidity group
        complication_count = procedure_df[procedure_df['complications'] == 1][col].value_counts()

        # Store the results for comorbidity presence (1) and absence (0)
        for value in [0, 1]:
            total = total_count.get(value, 0)
            comp_count = complication_count.get(value, 0)

            results.append({
                'Comorbidity': f"{col}_{value}",
                'Patients': f"{total} ({total / len(procedure_df) * 100:.2f}%)",
                'Patients w/ Complication': format_count_proportion(comp_count, total)
            })

    # Create a DataFrame from the results
    results_df = pd.DataFrame(results)

    # Setting 'Comorbidity' as the index
    results_df.set_index('Comorbidity', inplace=True)

    return results_df

table_4_counts = get_table_4_counts(procedure_df)
table_4_counts

In [ ]:
def get_odd_ratios_table_4():
    or_df4 = procedure_df[comorbidity_columns + ['complications'] + table_4_adjusted]
    or_df4 = condense_AGE(or_df4) 

    # Number of rows before dropping NaNs
    rows_before = or_df4.shape[0]
    # Drop rows with NaN values
    or_df4 = or_df4.dropna()
    # Number of rows after dropping NaNs
    rows_after = or_df4.shape[0]

    # Calculate and print the number of rows dropped
    rows_dropped = rows_before - rows_after
    print(f"Number of discharges dropped: {rows_dropped}")

    # Convert only comorbidity_columns to dummy variables
    or_df4 = pd.get_dummies(or_df4, columns=comorbidity_columns + table_4_adjusted)
    return or_df4

or_df4 = get_odd_ratios_table_4()
or_df4

In [ ]:
def get_covariate_columns(or_df):
    covariate_columns = [col for col in or_df.columns if col.startswith(tuple([prefix + '_' for prefix in table_4_adjusted]))]

    # Extracting the unique prefixes (like 'FEMALE', 'AGE', 'RACE')
    prefixes = set(col.split('_')[0] for col in covariate_columns)

    # Finding the first instance of each prefix
    first_instances = {prefix: next(col for col in covariate_columns if col.startswith(prefix)) for prefix in prefixes}
    
    # Hardcoding the replacement for the 'AGE' prefix
    first_instances['AGE'] = 'AGE_35-49'

    # Exclude the first instances to get the adjusted covariates
    adjusted_covariates = [col for col in covariate_columns if col not in first_instances.values()]

    return adjusted_covariates

covariate_columns_table_4 = get_covariate_columns(or_df4)
covariate_columns_table_4

In [ ]:
def get_variables_to_exclude(df, threshold):
    exclude_dict = {}
    for col in df.columns:
        true_count = df[col].sum()
        if true_count < threshold:
            exclude_dict[col] = true_count
    return exclude_dict

# Example usage
threshold = 11  # Set your threshold here
variables_to_exclude_table_4 = get_variables_to_exclude(or_df4, threshold)

print(f"excluding: {variables_to_exclude_table_4}")
print("______________________________________")

# todo: exlude "perfect prediction" or "complete separation" variables such as CMR_CANCER_LYMPH where 0 complications exist
      
def calculate_logistic_regression_with_reference(df, dependent_var, independent_vars_prefixes, covariates, variables_to_exclude):
    results = {}

    for prefix in independent_vars_prefixes:
        reference_var = f"{prefix}_0"
        target_var = f"{prefix}_1"

        # Check if the target variable is in the exclusion list
        if target_var not in variables_to_exclude:
            try:
                # Define X and y
                X = df[[target_var] + covariates].astype(float)
                y = df[dependent_var].astype(float)

                X = sm.add_constant(X)
                model = sm.Logit(y, X)
                fitted_model = model.fit(maxiter=1000)

                odds_ratio = np.exp(fitted_model.params[target_var]).round(2)
                conf = fitted_model.conf_int().loc[target_var]
                ci_lower, ci_upper = np.exp(conf[0]).round(2), np.exp(conf[1]).round(2)
                p_value = fitted_model.pvalues[target_var].round(3)

                results[f"{prefix}_1"] = {'OR': odds_ratio, '2.5%': ci_lower, '97.5%': ci_upper, 'p-value': p_value}
            except Exception as e:
                print(f"Error with variable {prefix}: {e}")

    results_df = pd.DataFrame(results).T
    results_df.index.name = 'Comorbidity'

    results_df['OR (95% CI)'] = results_df.apply(
        lambda row: f"{row['OR']} ({row['2.5%']} - {row['97.5%']})", axis=1
    )
    final_df = results_df[['OR (95% CI)', 'p-value']]

    return final_df


#variables_to_exclude = get_variables_to_exclude(or_df4, 2)  # Assume this is your exclusion list
logistic_regression_results_table_4 = calculate_logistic_regression_with_reference(or_df4, 
                                                                           dependent_var, 
                                                                           comorbidity_columns, 
                                                                           covariate_columns_table_4, 
                                                                           variables_to_exclude_table_4)
logistic_regression_results_table_4

In [ ]:
table_4 = pd.merge(table_4_counts, logistic_regression_results_table_4, on='Comorbidity', how='left')
table_4

In [ ]:
table_4.to_csv(f"{data_folder_name}/{researcher}_{table_4_name}")

# Table 5: Surgical parameters

In [ ]:
def add_surgical_params(df):
    # Helper function to check if a code starts with any prefix in a list
    def starts_with_any(code, prefixes):
        return any(code.startswith(prefix) for prefix in prefixes) if code is not None else False

    # List of procedure columns to check
    procedure_columns = [f'I10_PR{n}' for n in range(1, 25)]

    # Function to check across multiple columns
    def check_procedures(row, prefixes):
        return any(starts_with_any(row[col], prefixes) for col in procedure_columns)

    # Independent variables based on new surgical parameters
    df['bilateral'] = df.apply(lambda row: check_procedures(row, bilateral), axis=1).astype(int)
    df['unicondylar'] = df.apply(lambda row: check_procedures(row, unicondylar), axis=1).astype(int)
    df['patellofemoral'] = df.apply(lambda row: check_procedures(row, patellofemoral), axis=1).astype(int)
    
    df['cemented'] = df.apply(lambda row: check_procedures(row, cemented), axis=1).astype(int)
    df['uncemented'] = df.apply(lambda row: check_procedures(row, uncemented), axis=1).astype(int)
    
    df['synthetic'] = df.apply(lambda row: check_procedures(row, synthetic), axis=1).astype(int)
    df['autologous'] = df.apply(lambda row: check_procedures(row, autologous), axis=1).astype(int)
    df['non_autologous'] = df.apply(lambda row: check_procedures(row, non_autologous), axis=1).astype(int)
    
    df['articulating_spacer_yes'] = df.apply(lambda row: check_procedures(row, articulating_spacer_yes), axis=1).astype(int)

    # Covariates:
    df['comorbidity'] = df[comorbidity_columns].any(axis=1).astype(int)  # create dummy variable representing all comorbidities
    df = condense_AGE(df)  # convert 'AGE' into a dummy variable
    
    return df 

procedure_df = add_surgical_params(procedure_df)

procedure_df

In [ ]:
def check_surgical_partitions(procedure_df, partitions):
    # Extract the specified columns into a separate DataFrame
    subset_df = procedure_df[partitions]

    # Count the number of rows where the sum of all columns is not 1
    non_partition_count = (subset_df.sum(axis=1) != 1).sum()
    
    portion = 100 * round(non_partition_count / len(subset_df), 4)

    print(f"Number of rows where columns do not form a partition: {non_partition_count} ({portion}%)")
    return subset_df[(subset_df.sum(axis=1) != 1)]

# Example usage
check_surgical_partitions(procedure_df, ['bilateral', 'unicondylar', 'patellofemoral'])
check_surgical_partitions(procedure_df, ['cemented', 'uncemented'])
check_surgical_partitions(procedure_df, ['synthetic', 'autologous', 'non_autologous'])
check_surgical_partitions(procedure_df, ['articulating_spacer_yes'])  # For binary, only one column needed

In [ ]:
def add_patient_characteristics(table, procedure_df):
    """
    Add patient characteristics to a table.

    Parameters:
    - table (pd.DataFrame): The table to which the patient characteristics will be added.
    - procedure_df (pd.DataFrame): The DataFrame containing procedure data.
    - characteristics (list): A list of characteristics to be included.

    Returns:
    - pd.DataFrame: The updated table with patient characteristics.
    """
    
    non_condensed_vars = [col for col in surgical_independent_variables_with_reference]
    
    
    for col in non_condensed_vars:
        for value in procedure_df[col].unique():
            count_procedure = procedure_df[procedure_df[col] == value].shape[0]
            if count_procedure > 0:
                table.loc[f'{col}_{value}', 'procedure'] = f"{count_procedure} ({count_procedure/len(procedure_df):.2%})"
    return table

def add_complication_data(table, procedure_df):
    """
    Add complication data to a table.

    Parameters:
    - table (pd.DataFrame): The table to which the complication data will be added.
    - procedure_df (pd.DataFrame): The DataFrame containing procedure data, including complications.
    - characteristics (list): A list of characteristics to consider for complication data.

    Returns:
    - pd.DataFrame: The updated table with complication data.
    """
    non_condensed_vars = [col for col in surgical_independent_variables_with_reference]

    for col in non_condensed_vars:
        complication_counts = procedure_df.groupby(col)['complications'].sum()
        total_counts = procedure_df[col].value_counts()
        complication_proportion = complication_counts / total_counts

        for value, count in complication_counts.items():
            if total_counts.get(value, 0) > 0:
                proportion = complication_proportion.get(value, 0)
                table.loc[f'{col}_{value}', 'complication'] = f"{count} ({proportion:.2%})"
    return table

def get_table_5_counts(procedure_df, ):
    """
    Create a table with patient characteristics and complication information.

    Parameters:
    - procedure_df (pd.DataFrame): The DataFrame containing procedure data, including complications.
    - condensed (list): A list of condensed variables to consider.

    Returns:
    - pd.DataFrame: The table containing patient characteristics and complication information.
    """
    table = pd.DataFrame(columns=['procedure', 'complication']).astype('object')
    #table.loc['total count', 'procedure'] = f"{len(procedure_df)} ({len(procedure_df)/len(procedure_df):.2%})"
    #age_mean_std_procedure = f"{procedure_df['AGE'].mean():.2f}, {procedure_df['AGE'].std():.2f}"
    #table.loc['AGE (mean, std)', 'procedure'] = age_mean_std_procedure
    

    #table = add_complication_summary(table, procedure_df)
    table = add_patient_characteristics(table, procedure_df)
    table = add_complication_data(table, procedure_df)

    #for condensed_variable in condensed_table_5:
     #   if condensed_variable == 'AGE':
      #      table = add_AGE_table_3(table)

    # Split the table into the first row and the rest
    first_row = table.iloc[:2]
    rest_of_table = table.iloc[2:]

    # Sort the rest of the table based on the index (row labels) in ascending order
    rest_of_table_sorted = rest_of_table.copy().sort_index()

    # Concatenate the first row with the sorted rest of the table
    sorted_table = pd.concat([first_row, rest_of_table_sorted])

    return sorted_table

    
table_5_counts = get_table_5_counts(procedure_df)
table_5_counts

In [ ]:
def get_odd_ratios_table_5():

    or_df5 = procedure_df.copy() # already has independent vars + covariate vars (condensed age, comorbidity)

    # Create dummy variables for each surgical parameter:

    # or_df5 = add_surgical_params(or_df5)
    
    surg = [f"{dependent_var}"]+surgical_independent_variables+table_5_adjusted # relevant columns logistic regression 
    or_df5 = or_df5[surg]
    
    or_df5 = pd.get_dummies(or_df5, columns=or_df5.columns.drop('complications')) 
    
    return or_df5

or_df5 = get_odd_ratios_table_5()
or_df5

In [ ]:
def get_reference_variables(df, refrence_variables): 
    # Check if 'refrence_variables' exists globally
    if f'{refrence_variables}' in globals():
        return globals()[f'{refrence_variables}']

    # Identify all unique category prefixes (considering the part before the last underscore)
    unique_categories = set('_'.join(col.split('_')[:-1]) for col in df.columns if '_' in col)

    # Initialize a dictionary to store reference variables for each category
    reference_variables = {}

    for category in unique_categories:
        # Find the first dummy variable of each category to use as the reference
        reference_var = next((col for col in df.columns if col.startswith(category + '_')), None)
        
        if reference_var:
            reference_variables[category] = reference_var

    return reference_variables

def get_reference_variables(df, refrence_variables): 
    # Check if 'refrence_variables' exists globally
    if f'{refrence_variables}' in globals():
        return globals()[f'{refrence_variables}']

    # Identify all unique category prefixes (considering the part before the last underscore)
    unique_categories = set('_'.join(col.split('_')[:-1]) for col in df.columns if '_' in col)

    # Initialize a dictionary to store reference variables for each category
    reference_variables = {}

    for category in unique_categories:
        if category == 'AGE':
            # Hardcoding the reference variable for 'AGE' 
            reference_variables[category] = 'AGE_35-49'
        else:
            # For other categories, find the first dummy variable as the reference
            reference_var = next((col for col in df.columns if col.startswith(category + '_')), None)
            
            if reference_var:
                reference_variables[category] = reference_var

    return reference_variables


reference_vars = get_reference_variables(or_df5, 'refrence_variables_table_5')
print("Reference variables for each category:")
for category, ref_var in reference_vars.items():
    print(f"{category}: {ref_var}") # needed?

In [ ]:
def get_covariate_columns(or_df, adjusted):
    covariate_columns = [col for col in or_df.columns if col.startswith(tuple([prefix + '_' for prefix in adjusted]))]


    # Extracting the unique prefixes (like 'FEMALE', 'AGE', 'RACE')
    prefixes = set(col.split('_')[0] for col in covariate_columns)
    

    # Finding the first instance of each prefix
    first_instances = {prefix: next(col for col in covariate_columns if col.startswith(prefix)) for prefix in prefixes}

    # Exclude the first instances to get the adjusted covariates
    adjusted_covariates = [col for col in covariate_columns if col not in first_instances.values()]

    return adjusted_covariates

covariate_columns_table_5 = get_covariate_columns(or_df5, table_5_adjusted)
covariate_columns_table_5 # redundant (refactor get_reference_variables to include this logic) 

def get_covariate_columns(or_df, adjusted):
    covariate_columns = [col for col in or_df.columns if col.startswith(tuple([prefix + '_' for prefix in adjusted]))]

    # Extracting the unique prefixes (like 'FEMALE', 'AGE', 'RACE')
    prefixes = set(col.split('_')[0] for col in covariate_columns)
    
    # Finding the first instance of each prefix
    first_instances = {}
    for prefix in prefixes:
        if prefix == 'AGE':
            # Hardcoding the reference variable for 'AGE'
            first_instances[prefix] = 'AGE_35-49'
        else:
            first_instances[prefix] = next((col for col in covariate_columns if col.startswith(prefix)), None)

    # Exclude the first instances to get the adjusted covariates
    adjusted_covariates = [col for col in covariate_columns if col not in first_instances.values()]

    return adjusted_covariates

covariate_columns_table_5 = get_covariate_columns(or_df5, table_5_adjusted)
covariate_columns_table_5

In [ ]:
def get_variables_to_exclude(df, threshold):
    exclude_dict = {}
    for col in df.columns:
        true_count = df[col].sum()
        if true_count < threshold:
            exclude_dict[col] = true_count
    return exclude_dict

# Example usage
threshold = 11  # Set your threshold here
variables_to_exclude_table_5 = get_variables_to_exclude(or_df5, threshold)

print(f"excluding: {variables_to_exclude_table_5}")
print("______________________________________")

# todo: exlude "perfect prediction" or "complete separation" variables such as CMR_CANCER_LYMPH where 0 complications exist
      
def calculate_logistic_regression_with_reference(df, dependent_var, independent_vars_prefixes, covariates, variables_to_exclude):
    results = {}

    for prefix in independent_vars_prefixes:
        reference_var = f"{prefix}_0"
        target_var = f"{prefix}_1"

        # Check if the target variable is in the exclusion list
        if target_var not in variables_to_exclude:
            try:
                # Define X and y
                X = df[[target_var] + covariates].astype(float)
                y = df[dependent_var].astype(float)

                X = sm.add_constant(X)
                model = sm.Logit(y, X)
                fitted_model = model.fit(maxiter=1000)

                odds_ratio = np.exp(fitted_model.params[target_var]).round(2)
                conf = fitted_model.conf_int().loc[target_var]
                ci_lower, ci_upper = np.exp(conf[0]).round(2), np.exp(conf[1]).round(2)
                p_value = fitted_model.pvalues[target_var].round(3)

                results[f"{prefix}_1"] = {'OR': odds_ratio, '2.5%': ci_lower, '97.5%': ci_upper, 'p-value': p_value}
            except Exception as e:
                print(f"Error with variable {prefix}: {e}")

    results_df = pd.DataFrame(results).T
    #results_df.index.name = 'Comorbidity'

    results_df['OR (95% CI)'] = results_df.apply(
        lambda row: f"{row['OR']} ({row['2.5%']} - {row['97.5%']})", axis=1
    )
    final_df = results_df[['OR (95% CI)', 'p-value']]

    return final_df

#variables_to_exclude = get_variables_to_exclude(or_df4, 2)  # Assume this is your exclusion list
logistic_regression_results_table_5 = calculate_logistic_regression_with_reference(or_df5, 
                                                                           dependent_var, 
                                                                           surgical_independent_variables, 
                                                                           covariate_columns_table_5, 
                                                                           variables_to_exclude_table_5)
logistic_regression_results_table_5

In [ ]:
def get_table(counts_df, regression_df):

    # Reset the index of logistic_regression_results_table_5 so that the index becomes a column
    regression_df_reset = regression_df.reset_index()

    # Perform the merge
    # Note: Since table_5_counts is using index for merging, 
    # the resulting DataFrame will have the index from table_5_counts
    table_5 = pd.merge(counts_df, regression_df_reset, 
                       left_index=True, right_on='index', how='left')

    # Optionally, if you want to set 'index' column as the index of the merged DataFrame
    table_5.set_index('index', inplace=True)
    # Rename the index of table_5 to 'Parameter'
    table_5.rename_axis('Parameter', inplace=True)

    return table_5

table_5 = get_table(table_5_counts, logistic_regression_results_table_5)
table_5

In [ ]:
table_5.to_csv(f"{data_folder_name}/{researcher}_{table_5_name}")

In [ ]:
end_time = time.time()
total_time = end_time - start_time

# Convert seconds to minutes and seconds
minutes = int(total_time // 60)
seconds = int(total_time % 60)

print(f"Total execution time: {minutes} minutes, {seconds} seconds")

In [ ]:
# 1. Compartment
base_bilateral = ['0SR.C06', '0SRC.07', '0SR.C0J', '0SR.C0K', '0SR.D06', '0SR.D07', '0SR.D0J', '0SR.D0K'] 
base_unicondylar = ['0SR.C0L', '0SR.C0M', '0SR.D0L', '0SR.D0M']
base_patellofemoral = ['0SR.C0N', '0SR.D0N']

# 2. Cementation status
base_cemented = ['0SR.C069', '0SR.C0J9', '0SR.C0L9', '0SR.C0M9', '0SR.C0N9', '0SR.D069', '0SR.D0J9', '0SR.D0L9', '0SR.D0M9', '0SR.D0N9']
base_uncemented = ['0SR.C06A', '0SR.C0JA', '0SR.C0LA', '0SR.C0MA', '0SR.C0NA', '0SR.D06A', '0SR.D0JA', '0SR.D0LA', '0SR.D0MA', '0SR.D0NA']

# 3. Tissue substitute
base_synthetic = ['0SR.C06', '0SR.C0J', '0SR.C0L', '0SR.C0M', '0SR.C0N', '0SR.D06', '0SR.D0J', '0SR.D0L', '0SR.D0M', '0SR.D0N']
base_autologous = ['0SR.C07', '0SR.D07']
base_non_autologous = ['0SR.C0K', '0SR.D0K']

# 4. Articulating spacer
base_articulating_spacer_yes = ['0SR.C0E', '0SR.D0E']